In [1]:
import os
os.environ["USE_TF"] = "0"
# This disables TensorFlow usage in transformers and related libraries.


In [ ]:
#pip install docling langchain langchain-docling langchain-huggingface langchain-community sentence-transformers chromadb sqlalchemy psycopg2-binary spacy fastapi uvicorn gradio
#pip install --upgrade "transformers>=4.40"
#pip install pymupdf
#!python -m spacy download en_core_web_sm
#%pip uninstall numpy -y
#%pip install "numpy<2"


In [2]:
import torch
import numpy as np

# Check GPU availability
gpu_available = torch.cuda.is_available()
print(f"GPU Available: {gpu_available}")

# Print PyTorch version
print(f"PyTorch version: {torch.__version__}")

# Print NumPy version
print(f"NumPy version: {np.__version__}")

GPU Available: True
PyTorch version: 2.7.1+cu118
NumPy version: 2.2.6


In [3]:
import warnings


# --- Suppress warnings ---
def warn(*args, **kwargs):
    pass
warnings.warn = warn
warnings.filterwarnings("ignore")

In [4]:
import fitz  # PyMuPDF
from langchain_community.document_loaders import PyMuPDFLoader
from langchain_community.vectorstores.utils import filter_complex_metadata

pdf_path = "my_contract.pdf"  


def is_image_heavy(pdf_path, text_threshold=200, image_ratio_threshold=0.3):
    """
    Heuristically determine if a PDF is image-heavy.
    - text_threshold: minimum total text characters per page to count as text-based
    - image_ratio_threshold: fraction of pages with mostly images before we call it image-heavy
    """
    doc = fitz.open(pdf_path)
    total_pages = len(doc)
    image_pages = 0

    for page in doc:
        text = page.get_text("text")
        images = page.get_images()
        # if very little text and images exist, consider this page "image-heavy"
        if len(text.strip()) < text_threshold and len(images) > 0:
            image_pages += 1

    doc.close()

    ratio = image_pages / total_pages
    print(f"Image-heavy ratio: {ratio:.2f}")
    return ratio >= image_ratio_threshold

def load_contract(pdf_path, prefer_docling=True):
    """
    Auto-select loader based on document type.
    Returns a list of LangChain Document objects.
    """
    image_heavy = is_image_heavy(pdf_path) 
    if image_heavy:
        print("Detected image-heavy PDF, using DoclingLoader.")
        loader = DoclingLoader(pdf_path)
    else:
        print("Detected text-heavy PDF, using PyMuPDFLoader.")
        loader = PyMuPDFLoader(pdf_path)
        
    return loader

    # Step 3: Load and clean documents
    docs = loader.load()
    docs = filter_complex_metadata(docs)
    return docs



  
    


In [20]:
import hashlib
import json 
from pathlib import Path
from datetime import datetime
import string as str
INDEX_PATH = Path("./index.json")
metadata = []
def file_sha256(file_path):
    """Compute SHA256 hash of a file."""
    h = hashlib.sha256()
    with open(file_path, "rb") as f:
        for chunk in iter(lambda: f.read(8192), b""):
            h.update(chunk)
    return h.hexdigest()

def load_index():
    """Load index metadata from JSON file."""
    if INDEX_PATH.exists():
        return json.loads(INDEX_PATH.read_text())
    return {"documents": {}}

def save_index(index: dict):
    """Save index metadata to JSON file.using atomic save to avoid corruption."""
    tmp_path = INDEX_PATH.with_suffix(".tmp")
    tmp_path.write_text(json.dumps(index, indent=2), encoding="utf-8")
    tmp_path.replace(INDEX_PATH)  # atomic replace



def register_doc(doc_hash, pdf_path, persist_dir, extra=metadata):
    """Register a document hash with its metadata in the index."""
    idx = load_index()
    idx["documents"][doc_hash] = {
        "filename": filename,
        "persist_dir": persist_dir,
        "registered_at": datetime.utcnow().isoformat(),
        "extra": extra or {}
    }
    save_index(idx)
    
def is_doc_registered(doc_hash: str):
    """Check if a document hash is already registered in the index."""
    idx = load_index()
    return doc_hash in idx["documents"]

def get_doc_entry(doc_hash: str):
    """Get the metadata entry for a document hash."""
    idx = load_index()
    return idx["documents"].get(doc_hash, None)


In [6]:
import re
import spacy

nlp = spacy.load("en_core_web_sm")

#rule based (regex, spacy) extraction of entities
def rule_based_extract(text):
    results = {}
    confidence = {}

    # Example: effective date
    date_match = re.search(r"(?:effective\s+date\s*[:\-]?\s*)(\w+\s\d{1,2},\s\d{4})", text, re.IGNORECASE)
    if date_match:
        results["effective_date"] = date_match.group(1)
        confidence["effective_date"] = "high"
    else:
        confidence["effective_date"] = "low"

    # Example: party names (look for 'between X and Y')
    parties_match = re.search(r"This agreement.*between\s+(.+?)\s+and\s+(.+?)\.", text, re.IGNORECASE)
    if parties_match:
        results["party_names"] = [parties_match.group(1), parties_match.group(2)]
        confidence["party_names"] = "high"
    else:
        confidence["party_names"] = "low"

    return results, confidence




In [7]:
from transformers import pipeline

# Load the NER pipeline with a legal-friendly model
ner = pipeline("ner", model="Jean-Baptiste/roberta-large-ner-english", aggregation_strategy="simple", framework="pt")
def huggingface_fallback(text):
    entities = ner(text)
    # Post-process entities to fit your output format if needed
    return entities

Device set to use cuda:0


In [ ]:
docs = load_contract("my_contract.pdf")
print(len(docs))
print(docs[0].page_content[:300])

In [8]:
#%pip install langchain-openai

#llm fallback extraction of entities
from langchain_openai import ChatOpenAI
from langchain.prompts.chat import ChatPromptTemplate
from langchain.prompts import PromptTemplate
import json

prompt = ChatPromptTemplate.from_messages([
    ("system", "You are a legal document parser. Extract fields in JSON."),
    ("human", "Document text:\n\n{doc_text}\n\nReturn JSON with keys: party_names, effective_date, termination_date, jurisdiction, signatories.")
])

def llm_fallback(text):
    chain = prompt | llm
    response = chain.invoke({"doc_text": text})
    try:
        return json.loads(response.content)
    except Exception:
        return {"error": "Failed to parse JSON"}
    

In [9]:
#hybrid controller
def hybrid_extraction(text, threshold="high"):
    results, confidence = rule_based_extract(text)
    final = results.copy()
    
    # If any key has low confidence, use LLM fallback
    if any(c == "low" for c in confidence.values()):
        llm_results = llm_fallback(text)
        results.update(llm_results)
    
    return results

In [10]:
#chunking documents
from langchain_text_splitters import RecursiveCharacterTextSplitter

def chunk_docs(docs):
    splitter = RecursiveCharacterTextSplitter(
        chunk_size=1000,
        chunk_overlap=100  #contetxt overlap
        
    )
    return splitter.split_documents(docs)

In [ ]:
chunks = chunk_docs(docs)

print(f"Original docs: {len(docs)}")
print(f"Chunked docs: {len(chunks)}")

# Show first chunk
print(chunks[0].page_content[:200])
print(chunks[0].metadata)

In [11]:
#embedding model using huggingface transformers + auto moedel avoid sentence transformers dependency issues
from transformers import AutoTokenizer, AutoModel
import torch

class HFCustomEmbeddings:
    def __init__(self, model_name="sentence-transformers/all-mpnet-base-v2"):
        self.tokenizer = AutoTokenizer.from_pretrained(model_name)
        self.model = AutoModel.from_pretrained(model_name)
        self.device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
        self.model.to(self.device)

    def embed_text(self, text: str):
        import torch
        inputs = self.tokenizer(text, return_tensors="pt", truncation=True, padding=True).to(self.device)
        with torch.no_grad():
            outputs = self.model(**inputs)
            embeddings = outputs.last_hidden_state.mean(dim=1)
        return embeddings[0].cpu().numpy()

    # ✅ For LangChain compatibility
    def embed_documents(self, texts):
        return [self.embed_text(t) for t in texts]

    def embed_query(self, text):
        return self.embed_text(text)




In [14]:

from langchain_community.vectorstores.utils import filter_complex_metadata
chunks = chunk_docs(docs)
# Suppose chunks is your Docling split output
filtered_chunks = filter_complex_metadata(chunks)


NameError: name 'docs' is not defined

In [15]:
#testing similarity search with query
query = "What are the deposit amounts?"
results = vectorstore.similarity_search(query, k=3)

for r in results:
    print("----")
    print(r.page_content[:200])
    print(r.metadata)


NameError: name 'vectorstore' is not defined

In [16]:
from langchain_community.vectorstores import Chroma

emb_model = HFCustomEmbeddings("sentence-transformers/all-mpnet-base-v2")

vectorstore = Chroma.from_documents(
        documents=filtered_chunks,
        embedding=emb_model,
        persist_directory="./chroma_db" )#save locally

NameError: name 'filtered_chunks' is not defined

In [17]:
#addig

from transformers import AutoModelForSeq2SeqLM, AutoTokenizer, pipeline
from langchain_community.llms import HuggingFacePipeline
from langchain.chains import RetrievalQA

# Load FLAN-T5
model_name = "google/flan-t5-large"  #t5-large/t5-tglobal-base ideal if GPU is available
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSeq2SeqLM.from_pretrained(model_name)

# HF pipeline
pipe = pipeline(
    "text2text-generation",
    model=model,
    tokenizer=tokenizer,
    max_length=512
)

# Wrap in LangChain
local_llm = HuggingFacePipeline(pipeline=pipe)

# Retriever from your vector store
retriever = vectorstore.as_retriever(search_kwargs={"k": 3})

# Build RetrievalQA chain
qa_chain = RetrievalQA.from_chain_type(
    llm=local_llm,
    retriever=retriever,
    chain_type="stuff"  # simplest: just concatenate retrieved chunks
)



Device set to use cuda:0


NameError: name 'vectorstore' is not defined

In [21]:
# GRADIO UI (REVISED - cell [39])

import gradio as gr
from pathlib import Path

# Make sure qa_chain is accessible
qa_chain = None

def process_pdf(pdf_path):
    global qa_chain

    # Fix 1: Handle no file input
    if pdf_path is None:
        return "Error: Please upload a PDF file first."

    doc_hash = file_sha256(pdf_path)
    persist_dir = f"./chroma_db/{doc_hash}"
    filename = Path(pdf_path).name # Define filename here

    # This is the embedding model you initialized earlier
    # You may want to define it globally if it's not already
    emb_model = HFCustomEmbeddings("sentence-transformers/all-mpnet-base-v2")

    if is_doc_registered(doc_hash):
        entry = get_doc_entry(doc_hash)
        # Load the existing vectorstore
        vectorstore = Chroma(persist_directory=entry["persist_dir"], embedding_function=emb_model)
        
        # Fix 4: Re-create the QA chain for the cached document
        retriever = vectorstore.as_retriever(search_kwargs={"k": 3})
        qa_chain = RetrievalQA.from_chain_type(llm=local_llm, retriever=retriever, chain_type="stuff")
        
        return f"Document '{filename}' is already processed. You can now ask questions."

    # --- Processing for a new document ---
    
    # load_contract now correctly returns a loader object
    loader = load_contract(pdf_path) 
    docs = loader.load()

    # Filter metadata
    filtered_docs = filter_complex_metadata(docs)

    # Chunk
    chunks = chunk_docs(filtered_docs)
    
    # Embed and create vectorstore
    vectorstore = Chroma.from_documents(
        documents=chunks, 
        embedding=emb_model,
        persist_directory=persist_dir # Persist while creating
    )

    # Create QA chain
    retriever = vectorstore.as_retriever(search_kwargs={"k": 3})
    qa_chain = RetrievalQA.from_chain_type(llm=local_llm, retriever=retriever, chain_type="stuff")

    # Register the document after successful processing
    metadata = {
        "original_path": filename,
        "persist_dir": persist_dir,
        "num_chunks": len(chunks)
    }
    # Pass necessary variables to register_doc
    register_doc(doc_hash, pdf_path, persist_dir, filename, extra=metadata) # You need to adjust register_doc to accept filename

    return f"PDF '{filename}' processed. You can now ask questions."

def answer_query(query):
    if qa_chain is None:
        return "Please upload and process a PDF first."
    
    # Fix 5: Use .invoke() instead of the deprecated .run()
    # The output of RetrievalQA is a dictionary, we need the 'result' key.
    result = qa_chain.invoke({"query": query})
    return result['result']

# You will also need to slightly modify register_doc in cell [18]
# to accept 'filename' as a parameter
def register_doc(doc_hash, pdf_path, persist_dir, filename, extra=metadata): # Added filename
    """Register a document hash with its metadata in the index."""
    idx = load_index()
    idx["documents"][doc_hash] = {
        "filename": filename, # Use the passed filename
        "persist_dir": persist_dir,
        "registered_at": datetime.utcnow().isoformat(),
        "extra": extra or {}
    }
    save_index(idx)

In [22]:
with gr.Blocks() as demo:
    gr.Markdown("## 📄 Legal Document Q&A")
    with gr.Row():
        pdf_input = gr.File(label="Upload PDF", file_types=[".pdf"])
        upload_btn = gr.Button("Process Document")
    status = gr.Textbox(label="Status")
    query = gr.Textbox(label="Ask a question")
    answer = gr.Textbox(label="Answer")

    upload_btn.click(fn=process_pdf, inputs=pdf_input, outputs=status)
    query.submit(fn=answer_query, inputs=query, outputs=answer)

demo.launch()

* Running on local URL:  http://127.0.0.1:7861
* To create a public link, set `share=True` in `launch()`.


Image-heavy ratio: 0.00
Detected text-heavy PDF, using PyMuPDFLoader.


Token indices sequence length is longer than the specified maximum sequence length for this model (836 > 512). Running this sequence through the model will result in indexing errors
